In [1]:
cd ..

/Users/mgcrp/Desktop/RLIO Environment


In [2]:
import numpy as np
import pandas as pd

from tqdm import tqdm
from os import listdir, getcwd
from os.path import isfile, join

In [ ]:
%%time

for file in listdir( join(getcwd(), 'store_raw_data') ):
    df = pd.read_csv(f'store_raw_data/{file}', sep=';')
    print( file )
    df.curr_date = pd.to_datetime(df.curr_date, format='%d%b%Y')
    print( df.curr_date.min() )
    print( df.curr_date.max() )
    print('---')

MERGE_TABLE_STORE_185200.csv
2016-01-01 00:00:00
2019-12-30 00:00:00
---
MERGE_TABLE_STORE_146100.csv
2016-01-01 00:00:00
2019-12-30 00:00:00
---
MERGE_TABLE_STORE_183300.csv
2016-01-01 00:00:00
2019-12-30 00:00:00
---
MERGE_TABLE_STORE_211800.csv
2016-01-01 00:00:00
2019-12-30 00:00:00
---
MERGE_TABLE_STORE_35200.csv
2016-01-02 00:00:00
2019-12-30 00:00:00
---
MERGE_TABLE_STORE_412000.csv
2016-07-08 00:00:00
2019-12-30 00:00:00
---
MERGE_TABLE_STORE_140600.csv
2016-01-02 00:00:00
2019-12-30 00:00:00
---
MERGE_TABLE_STORE_365000.csv
2016-03-04 00:00:00
2019-12-30 00:00:00
---
MERGE_TABLE_STORE_364900.csv
2016-03-04 00:00:00
2019-12-30 00:00:00
---
MERGE_TABLE_STORE_375400.csv
2016-04-01 00:00:00
2019-12-30 00:00:00
---
MERGE_TABLE_STORE_44400.csv
2016-01-02 00:00:00
2019-12-30 00:00:00
---
MERGE_TABLE_STORE_22800.csv
2016-01-01 00:00:00
2019-12-30 00:00:00
---
MERGE_TABLE_STORE_422700.csv
2016-10-13 00:00:00
2019-12-30 00:00:00
---
MERGE_TABLE_STORE_140400.csv
2016-01-01 00:00:00
2019-

Все даты начала/конца разные

Будем работать с каждым файлом отдельно

In [3]:
%%time
df = pd.read_csv('store_raw_data/MERGE_TABLE_STORE_4600.csv', sep=';', decimal='.')
df.curr_date = pd.to_datetime(df.curr_date, format='%d%b%Y')
df.head(10)

CPU times: user 43.6 s, sys: 2.51 s, total: 46.1 s
Wall time: 49.9 s


,product_id,store_id,curr_date,s_qty,flg_spromo,stock
0,200,4600,2016-02-01,NaN,0,2.0
1,200,4600,2016-02-02,NaN,0,2.0
2,200,4600,2016-02-03,NaN,0,2.0
3,200,4600,2016-02-04,NaN,0,2.0
4,200,4600,2016-02-05,NaN,0,2.0
5,200,4600,2016-02-06,NaN,0,2.0
6,200,4600,2016-02-07,NaN,0,2.0
7,200,4600,2016-02-08,NaN,0,2.0
8,200,4600,2016-02-09,NaN,0,2.0
9,200,4600,2016-02-10,NaN,0,2.0


In [4]:
store = 4600

tmp = []

for sku in tqdm( df.product_id.unique() ):
    if len( pd.date_range(start = df[df.product_id == sku].curr_date.min(), end = df[df.product_id == sku].curr_date.max() ).difference(df[df.product_id == sku].curr_date) ):
        for date in pd.date_range(start = df[df.product_id == sku].curr_date.min(), end = df[df.product_id == sku].curr_date.max() ).difference(df[df.product_id == sku].curr_date):
            tmp.append(
                {
                    'product_id': sku,
                    'store_id': store,
                    'curr_date': date,
                    's_qty': 0,
                    'flg_spromo': 0,
                    'stock': np.nan
                }
            )

100%|██████████| 12554/12554 [10:00<00:00, 20.90it/s]


In [5]:
df_tmp = pd.DataFrame(tmp)
df_tmp

,product_id,store_id,curr_date,s_qty,flg_spromo,stock
0,23700,4600,2019-09-17,0,0,NaN
1,27600,4600,2019-09-18,0,0,NaN
2,27600,4600,2019-09-19,0,0,NaN
3,30300,4600,2019-08-25,0,0,NaN
4,35400,4600,2019-09-13,0,0,NaN
...,...,...,...,...,...,...
24747,19959600,4600,2019-09-15,0,0,NaN
24748,19959600,4600,2019-09-16,0,0,NaN
24749,19959600,4600,2019-09-17,0,0,NaN
24750,19959600,4600,2019-09-18,0,0,NaN


In [6]:
df = df.append(df_tmp, ignore_index=True)

In [7]:
for sku in tqdm( df.product_id.unique() ):
    if len( pd.date_range(start = df[df.product_id == sku].curr_date.min(), end = df[df.product_id == sku].curr_date.max() ).difference(df[df.product_id == sku].curr_date) ):
        print(sku)
        print(df[df.product_id == sku].curr_date.min())
        print(df[df.product_id == sku].curr_date.max())
        print(f'Missing: {pd.date_range(start = df[df.product_id == sku].curr_date.min(), end = df[df.product_id == sku].curr_date.max() ).difference(df[df.product_id == sku].curr_date)}')
        print('---')

100%|██████████| 12554/12554 [10:24<00:00, 20.10it/s]


In [8]:
df_sl = pd.read_csv('echelon_processed_data/mply_qty_normalized.csv', sep=';')
df_sl.curr_date = pd.to_datetime(df_sl.curr_date)
df_sl.head(10)

,product_id,store_id,curr_date,mply_qty
0,200,16500,2019-05-18,1
1,200,16500,2019-05-19,1
2,200,16500,2019-05-20,1
3,200,16500,2019-05-21,1
4,200,16500,2019-05-22,1
5,200,16500,2019-05-24,1
6,200,16500,2019-05-25,1
7,200,16500,2019-05-26,1
8,200,16500,2019-05-27,1
9,200,16500,2019-05-28,1


In [9]:
df_norm = pd.merge(
    how='left',
    left=df,
    left_on=['product_id', 'store_id', 'curr_date'],
    right=df_sl,
    right_on=['product_id', 'store_id', 'curr_date']
)
df_norm.head(10)

,product_id,store_id,curr_date,s_qty,flg_spromo,stock,mply_qty
0,200,4600,2016-02-01,NaN,0,2.0,NaN
1,200,4600,2016-02-02,NaN,0,2.0,NaN
2,200,4600,2016-02-03,NaN,0,2.0,NaN
3,200,4600,2016-02-04,NaN,0,2.0,NaN
4,200,4600,2016-02-05,NaN,0,2.0,NaN
5,200,4600,2016-02-06,NaN,0,2.0,NaN
6,200,4600,2016-02-07,NaN,0,2.0,NaN
7,200,4600,2016-02-08,NaN,0,2.0,NaN
8,200,4600,2016-02-09,NaN,0,2.0,NaN
9,200,4600,2016-02-10,NaN,0,2.0,NaN


In [10]:
df_sl = pd.read_csv('echelon_processed_data/lead_time_normalized.csv', sep=';')
df_sl.curr_date = pd.to_datetime(df_sl.curr_date)
df_sl.head(10)

,product_id,store_id,curr_date,lead_time
0,200,16500,2019-05-18,1
1,200,16500,2019-05-19,1
2,200,16500,2019-05-20,1
3,200,16500,2019-05-21,1
4,200,16500,2019-05-22,1
5,200,16500,2019-05-24,1
6,200,16500,2019-05-25,1
7,200,16500,2019-05-26,1
8,200,16500,2019-05-27,1
9,200,16500,2019-05-28,1


In [11]:
df_norm = pd.merge(
    how='left',
    left=df_norm,
    left_on=['product_id', 'store_id', 'curr_date'],
    right=df_sl,
    right_on=['product_id', 'store_id', 'curr_date']
)
df_norm.head(10)

,product_id,store_id,curr_date,s_qty,flg_spromo,stock,mply_qty,lead_time
0,200,4600,2016-02-01,NaN,0,2.0,NaN,NaN
1,200,4600,2016-02-02,NaN,0,2.0,NaN,NaN
2,200,4600,2016-02-03,NaN,0,2.0,NaN,NaN
3,200,4600,2016-02-04,NaN,0,2.0,NaN,NaN
4,200,4600,2016-02-05,NaN,0,2.0,NaN,NaN
5,200,4600,2016-02-06,NaN,0,2.0,NaN,NaN
6,200,4600,2016-02-07,NaN,0,2.0,NaN,NaN
7,200,4600,2016-02-08,NaN,0,2.0,NaN,NaN
8,200,4600,2016-02-09,NaN,0,2.0,NaN,NaN
9,200,4600,2016-02-10,NaN,0,2.0,NaN,NaN


In [12]:
df_sl = pd.read_csv('echelon_processed_data/batch_size_normalized.csv', sep=';')
df_sl.curr_date = pd.to_datetime(df_sl.curr_date)
df_sl.head(10)

,product_id,store_id,curr_date,batch_size
0,200,16500,2019-05-18,1
1,200,16500,2019-05-19,1
2,200,16500,2019-05-20,1
3,200,16500,2019-05-21,1
4,200,16500,2019-05-22,1
5,200,16500,2019-05-24,1
6,200,16500,2019-05-25,1
7,200,16500,2019-05-26,1
8,200,16500,2019-05-27,1
9,200,16500,2019-05-28,1


In [13]:
df_norm = pd.merge(
    how='left',
    left=df_norm,
    left_on=['product_id', 'store_id', 'curr_date'],
    right=df_sl,
    right_on=['product_id', 'store_id', 'curr_date']
)
df_norm.head(10)

,product_id,store_id,curr_date,s_qty,flg_spromo,stock,mply_qty,lead_time,batch_size
0,200,4600,2016-02-01,NaN,0,2.0,NaN,NaN,NaN
1,200,4600,2016-02-02,NaN,0,2.0,NaN,NaN,NaN
2,200,4600,2016-02-03,NaN,0,2.0,NaN,NaN,NaN
3,200,4600,2016-02-04,NaN,0,2.0,NaN,NaN,NaN
4,200,4600,2016-02-05,NaN,0,2.0,NaN,NaN,NaN
5,200,4600,2016-02-06,NaN,0,2.0,NaN,NaN,NaN
6,200,4600,2016-02-07,NaN,0,2.0,NaN,NaN,NaN
7,200,4600,2016-02-08,NaN,0,2.0,NaN,NaN,NaN
8,200,4600,2016-02-09,NaN,0,2.0,NaN,NaN,NaN
9,200,4600,2016-02-10,NaN,0,2.0,NaN,NaN,NaN


In [14]:
df_sl = pd.read_csv('echelon_processed_data/service_level_normalized.csv', sep=';')
df_sl.curr_date = pd.to_datetime(df_sl.curr_date)
df_sl.head(10)

,product_id,store_id,curr_date,service_level
0,200,16500,2019-05-18,0.81
1,200,16500,2019-05-19,0.81
2,200,16500,2019-05-20,0.90
3,200,16500,2019-05-21,0.81
4,200,16500,2019-05-22,0.95
5,200,16500,2019-05-24,0.81
6,200,16500,2019-05-25,0.81
7,200,16500,2019-05-26,0.81
8,200,16500,2019-05-27,0.81
9,200,16500,2019-05-28,0.81


In [15]:
df_norm = pd.merge(
    how='left',
    left=df_norm,
    left_on=['product_id', 'store_id', 'curr_date'],
    right=df_sl,
    right_on=['product_id', 'store_id', 'curr_date']
)
df_norm.head(10)

,product_id,store_id,curr_date,s_qty,flg_spromo,stock,mply_qty,lead_time,batch_size,service_level
0,200,4600,2016-02-01,NaN,0,2.0,NaN,NaN,NaN,NaN
1,200,4600,2016-02-02,NaN,0,2.0,NaN,NaN,NaN,NaN
2,200,4600,2016-02-03,NaN,0,2.0,NaN,NaN,NaN,NaN
3,200,4600,2016-02-04,NaN,0,2.0,NaN,NaN,NaN,NaN
4,200,4600,2016-02-05,NaN,0,2.0,NaN,NaN,NaN,NaN
5,200,4600,2016-02-06,NaN,0,2.0,NaN,NaN,NaN,NaN
6,200,4600,2016-02-07,NaN,0,2.0,NaN,NaN,NaN,NaN
7,200,4600,2016-02-08,NaN,0,2.0,NaN,NaN,NaN,NaN
8,200,4600,2016-02-09,NaN,0,2.0,NaN,NaN,NaN,NaN
9,200,4600,2016-02-10,NaN,0,2.0,NaN,NaN,NaN,NaN


In [24]:
# df_norm.to_csv('store_processed_data/STORE_4600.csv', index=False)

In [25]:
# %%time
# df_norm.sort_values(by=['product_id', 'curr_date'], ascending=[True, True], inplace=True)
# df_norm.head(10)

CPU times: user 4.6 s, sys: 5.7 s, total: 10.3 s
Wall time: 13.1 s


,product_id,store_id,curr_date,s_qty,flg_spromo,stock,mply_qty,lead_time,batch_size,service_level
0,200,4600,2016-02-01,NaN,0,2.0,NaN,NaN,NaN,NaN
1,200,4600,2016-02-02,NaN,0,2.0,NaN,NaN,NaN,NaN
2,200,4600,2016-02-03,NaN,0,2.0,NaN,NaN,NaN,NaN
3,200,4600,2016-02-04,NaN,0,2.0,NaN,NaN,NaN,NaN
4,200,4600,2016-02-05,NaN,0,2.0,NaN,NaN,NaN,NaN
5,200,4600,2016-02-06,NaN,0,2.0,NaN,NaN,NaN,NaN
6,200,4600,2016-02-07,NaN,0,2.0,NaN,NaN,NaN,NaN
7,200,4600,2016-02-08,NaN,0,2.0,NaN,NaN,NaN,NaN
8,200,4600,2016-02-09,NaN,0,2.0,NaN,NaN,NaN,NaN
9,200,4600,2016-02-10,NaN,0,2.0,NaN,NaN,NaN,NaN


In [26]:
# df_norm.to_csv('store_processed_data/STORE_4600.csv', index=False)

In [27]:
# print(
#     df_norm[
#         (df_norm.product_id == 200) &
#         (df_norm.store_id == 4600) &
#         pd.notnull(df_norm.mply_qty)
#     ].curr_date.min()
# )

# print(
#     df_norm[
#         (df_norm.product_id == 200) &
#         (df_norm.store_id == 4600) &
#         pd.notnull(df_norm.lead_time)
#     ].curr_date.min()
# )

# print(
#     df_norm[
#         (df_norm.product_id == 200) &
#         (df_norm.store_id == 4600) &
#         pd.notnull(df_norm.batch_size)
#     ].curr_date.min()
# )

# print(
#     df_norm[
#         (df_norm.product_id == 200) &
#         (df_norm.store_id == 4600) &
#         pd.notnull(df_norm.service_level)
#     ].curr_date.min()
# )

# print(
#     df_norm[
#         (df_norm.product_id == 200) &
#         (df_norm.store_id == 4600) &
#         pd.notnull(df_norm.mply_qty) &
#         pd.notnull(df_norm.lead_time) &
#         pd.notnull(df_norm.batch_size) &
#         pd.notnull(df_norm.service_level)
#     ].curr_date.min()
# )

NaT
NaT
NaT
NaT
NaT


In [36]:
# df_norm[
#     (df_norm.product_id == 20030100) &
#     (df_norm.store_id == 4600) &
#     pd.notnull(df_norm.lead_time)
# ]

,product_id,store_id,curr_date,s_qty,flg_spromo,stock,mply_qty,lead_time,batch_size,service_level


In [39]:
# df_norm[
#     (df_norm.store_id == 4600) &
#     (df_norm.product_id == 200) &
#     pd.notnull(df_norm.s_qty)
# ]

,product_id,store_id,curr_date,s_qty,flg_spromo,stock,mply_qty,lead_time,batch_size,service_level
56,200,4600,2016-03-28,1.0,0,2.0,NaN,NaN,NaN,NaN
58,200,4600,2016-03-30,0.0,0,1.0,NaN,NaN,NaN,NaN
102,200,4600,2016-05-13,1.0,0,2.0,NaN,NaN,NaN,NaN
104,200,4600,2016-05-15,0.0,0,1.0,NaN,NaN,NaN,NaN
148,200,4600,2016-06-28,1.0,0,2.0,NaN,NaN,NaN,NaN
261,200,4600,2016-10-19,1.0,0,1.0,NaN,NaN,NaN,NaN
269,200,4600,2016-10-27,0.0,0,0.0,NaN,NaN,NaN,NaN
288,200,4600,2016-11-15,1.0,0,2.0,NaN,NaN,NaN,NaN
502,200,4600,2017-06-17,1.0,0,1.0,NaN,NaN,NaN,NaN
524,200,4600,2017-07-09,0.0,0,0.0,NaN,NaN,NaN,NaN


In [17]:
df_tmp = df_norm.dropna(subset=['mply_qty', 'lead_time', 'batch_size', 'service_level'], how='all')
df_tmp

,product_id,store_id,curr_date,s_qty,flg_spromo,stock,mply_qty,lead_time,batch_size,service_level
20617,23100,4600,2019-05-18,NaN,0,2.0,1.0,1.0,1.0,0.95
20618,23100,4600,2019-05-19,NaN,0,2.0,1.0,1.0,1.0,0.95
20619,23100,4600,2019-05-20,NaN,0,2.0,1.0,1.0,1.0,0.90
20620,23100,4600,2019-05-21,NaN,0,2.0,1.0,1.0,1.0,0.81
20621,23100,4600,2019-05-22,NaN,0,2.0,1.0,1.0,1.0,0.95
...,...,...,...,...,...,...,...,...,...,...
13607707,19901700,4600,2019-06-22,NaN,0,1.0,1.0,1.0,1.0,0.50
13607708,19901700,4600,2019-06-23,NaN,0,1.0,1.0,1.0,1.0,0.50
13607709,19901700,4600,2019-06-24,NaN,0,1.0,1.0,1.0,1.0,0.50
13607710,19901700,4600,2019-06-25,NaN,0,1.0,1.0,1.0,1.0,0.50


In [43]:
list_exist = []
list_empty = []

for sku in tqdm( df_norm.product_id.unique() ):
    if len( df_tmp[df_tmp.product_id == sku] ):
        list_exist.append(sku)
    else:
        list_empty.append(sku)

100%|██████████| 12554/12554 [00:11<00:00, 1046.90it/s]


In [44]:
len(list_exist)

5089

In [45]:
len(list_empty)

7465

In [46]:
list_exist[:10]

[23100, 23700, 25100, 25200, 25600, 25700, 27600, 27800, 28000, 29400]

In [47]:
# print(
#     df_norm[
#         (df_norm.product_id == 23100) &
#         (df_norm.store_id == 4600) &
#         pd.notnull(df_norm.mply_qty)
#     ].curr_date.min()
# )

# print(
#     df_norm[
#         (df_norm.product_id == 23100) &
#         (df_norm.store_id == 4600) &
#         pd.notnull(df_norm.lead_time)
#     ].curr_date.min()
# )

# print(
#     df_norm[
#         (df_norm.product_id == 23100) &
#         (df_norm.store_id == 4600) &
#         pd.notnull(df_norm.batch_size)
#     ].curr_date.min()
# )

# print(
#     df_norm[
#         (df_norm.product_id == 23100) &
#         (df_norm.store_id == 4600) &
#         pd.notnull(df_norm.service_level)
#     ].curr_date.min()
# )

# print(
#     df_norm[
#         (df_norm.product_id == 23100) &
#         (df_norm.store_id == 4600) &
#         pd.notnull(df_norm.mply_qty) &
#         pd.notnull(df_norm.lead_time) &
#         pd.notnull(df_norm.batch_size) &
#         pd.notnull(df_norm.service_level)
#     ].curr_date.min()
# )

2019-05-18 00:00:00
2019-05-18 00:00:00
2019-05-18 00:00:00
2019-05-18 00:00:00
2019-05-18 00:00:00


In [19]:
df_norm = df_norm[ df_norm.product_id.isin(df_tmp.product_id.unique().tolist()) ]

In [50]:
df_accepted = pd.DataFrame()

for sku in tqdm( df_norm.product_id.unique() ):
    df_accepted = df_accepted.append(
        df_norm[
            (df_norm.product_id == sku) &
            (
                df_norm.curr_date >= df_norm[
                    (df_norm.product_id == sku) &
                    pd.notnull(df_norm.mply_qty) &
                    pd.notnull(df_norm.lead_time) &
                    pd.notnull(df_norm.batch_size) &
                    pd.notnull(df_norm.service_level)
                ].curr_date.min()
            )
        ], ignore_index=True
    )

100%|██████████| 5089/5089 [12:14<00:00,  6.93it/s]


In [25]:
df_norm.isnull().sum()

product_id       0
store_id         0
curr_date        0
s_qty            0
flg_spromo       0
stock            0
mply_qty         0
lead_time        0
batch_size       0
service_level    0
dtype: int64

In [24]:
df_norm.s_qty.fillna(0, inplace=True)
df_norm.stock.fillna(0, inplace=True)

df_norm.mply_qty.fillna(method='ffill', inplace=True)
df_norm.lead_time.fillna(method='ffill', inplace=True)
df_norm.batch_size.fillna(method='ffill', inplace=True)
df_norm.service_level.fillna(method='ffill', inplace=True)

df_norm.mply_qty.fillna(method='bfill', inplace=True)
df_norm.lead_time.fillna(method='bfill', inplace=True)
df_norm.batch_size.fillna(method='bfill', inplace=True)
df_norm.service_level.fillna(method='bfill', inplace=True)

In [26]:
df_norm

,product_id,store_id,curr_date,s_qty,flg_spromo,stock,mply_qty,lead_time,batch_size,service_level
19923,23100,4600,2017-06-23,0.0,0,0.0,1.0,1.0,1.0,0.95
19924,23100,4600,2017-06-24,0.0,0,1.0,1.0,1.0,1.0,0.95
19925,23100,4600,2017-06-25,0.0,0,1.0,1.0,1.0,1.0,0.95
19926,23100,4600,2017-06-26,0.0,0,1.0,1.0,1.0,1.0,0.95
19927,23100,4600,2017-06-27,0.0,0,1.0,1.0,1.0,1.0,0.95
...,...,...,...,...,...,...,...,...,...,...
13641807,19901600,4600,2019-08-31,0.0,0,0.0,1.0,1.0,1.0,0.45
13641808,19901600,4600,2019-09-01,0.0,0,0.0,1.0,1.0,1.0,0.45
13641809,19901600,4600,2019-09-02,0.0,0,0.0,1.0,1.0,1.0,0.45
13641810,19901600,4600,2019-09-03,0.0,0,0.0,1.0,1.0,1.0,0.45


In [29]:
df_norm.loc[df_norm.stock < 0, 'stock'] = 0

In [30]:
df_norm.loc[df_norm.stock < 0, 'stock']

Series([], Name: stock, dtype: float64)

In [31]:
df_norm.to_csv('store_processed_data/STORE_4600.csv', index=False)

In [62]:
df_norm.groupby(by='product_id').curr_date.min()

product_id
23100      2019-05-18
23700      2019-05-18
25100      2019-05-18
25200      2019-05-18
25600      2019-05-18
              ...    
19900300   2019-06-22
19900700   2019-06-21
19900800   2019-06-21
19901600   2019-06-21
19901700   2019-06-21
Name: curr_date, Length: 5089, dtype: datetime64[ns]

In [65]:
df_sl = pd.read_csv('echelon_processed_data/service_level_normalized.csv', sep=';')
df_sl.curr_date.min()

'2019-05-18 00:00:00'

In [66]:
df_sl = pd.read_csv('echelon_processed_data/batch_size_normalized.csv', sep=';')
df_sl.curr_date.min()

'2019-05-18 00:00:00'

In [67]:
df_sl = pd.read_csv('echelon_processed_data/mply_qty_normalized.csv', sep=';')
df_sl.curr_date.min()

'2019-05-18 00:00:00'

In [68]:
df_sl = pd.read_csv('echelon_processed_data/lead_time_normalized.csv', sep=';')
df_sl.curr_date.min()

'2019-05-18 00:00:00'

In [71]:
df_sl = pd.read_csv('echelon_raw_data/echelon_1_sl.csv', sep=';')
df_sl.date_from = pd.to_datetime(df_sl.date_from, format='%d%b%Y')
df_sl.date_to = pd.to_datetime(df_sl.date_to, format='%d%b%Y')
df_sl.date_from.min()

Timestamp('2019-05-18 00:00:00')